Imports

In [10]:
import pandas as pd
import random
import numpy as np
import warnings

Global functions

In [3]:
def get_accident_df(subset="standard_small", n_rows=10000, is_refined=False):
    warnings.warn("This function is outdated and should not be used.")
    if is_refined:
        df = get_accident_df(subset=subset, n_rows=n_rows, is_refined=False)
        return refine_df(df)

    if subset == "standard_small":
        return pd.read_csv("DataSubsetRaw.csv")
    if subset == "new":
        n = 3513617 #number of rows in the dataset
        s = n_rows #number of rows to sample
        skip = sorted(random.sample(range(1, n+1), n-s)) #rows to be skipped
        return pd.read_csv("US_Accidents_June20.csv", skiprows=skip)
    if subset == "all":
        return pd.read_csv("US_Accidents_June20.csv")

Internal functions

In [12]:
def refine_df(df):
    #Modify the df in various ways
    return df

Testing

In [4]:
df = pd.read_csv("DataSubsetRaw.csv")

In [ ]:
def convert_to_metric(df):
    df["Distance(km)"] = df["Distance(mi)"] * 1.60934
    df["Temperature(C)"] = (df["Temperature(F)"] - 32) * 5 / 9
    return df